# Models results

In [1]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
import pandas as pd
from pathlib import PurePosixPath
from src.definitions import ROOT_DIR

from typing import List, Optional
from src.models.baseline import FPCBOWResNet152ConcatBaselineModel
from src.models.bert import FPBERTResNet152ConcatModel, FPBERTResNet152WeightedConcatModel
from src.models.clip import FPCLIPConcatModel, FPCLIPWeightedConcatModel
from src.models.base_model import FoodPricingBaseModel

### Create the table storing all the results

In [2]:
def store_results(
    df: pd.DataFrame,
    submission_frame: pd.DataFrame,
    model_name: str,
    run: Optional[str] = "",
) -> None:
    df.loc[(model_name, run), ["MSE", "RMSE", "MAE"]] = (
        mean_squared_error(
            y_true=submission_frame["true"] / 100,
            y_pred=submission_frame["pred"] / 100,
            squared=True,
        ),
        mean_squared_error(
            y_true=submission_frame["true"] / 100,
            y_pred=submission_frame["pred"] / 100,
            squared=False,
        ),
        mean_absolute_error(
            y_true=submission_frame["true"] / 100, y_pred=submission_frame["pred"] / 100
        ),
    )

In [3]:
results = pd.DataFrame(columns=["Model", "Run", "MSE", "RMSE", "MAE"])
results = results.set_index(["Model", "Run"])

In [4]:
results

,,MSE,RMSE,MAE
Model,Run,,,


### Mean Baseline

In [5]:
submissions_fpath = PurePosixPath(ROOT_DIR).joinpath("submissions", "2022-08-27T17:04:06+00:00")

mean_baseline_fname = submissions_fpath.joinpath("FPMeanBaselineModel.csv")

sub_mean_baseline = pd.read_csv(mean_baseline_fname,index_col=0,header=0)

In [6]:
sub_mean_baseline

,true,pred
item_id,,
bergamo_alzano-lombardo_pony-burger_5,850.0,659.82497
bergamo_centro-storico_bar-mada_1,1490.0,659.82497
bergamo_centro-storico_bar-mada_14,500.0,659.82497
bergamo_centro-storico_bar-mada_17,1240.0,659.82497
bergamo_centro-storico_bar-mada_24,490.0,659.82497
...,...,...
roma_villaggio-olimpico_enso_7,350.0,659.82497
roma_villaggio-olimpico_enso_75,2000.0,659.82497
roma_villaggio-olimpico_enso_78,1000.0,659.82497


In [7]:
store_results(results, sub_mean_baseline, "FPMeanBaselineModel")
results

,,MSE,RMSE,MAE
Model,Run,,,
FPMeanBaselineModel,,37.629804,6.134314,4.201829


### XGB Models

In [8]:
xgb_bert_resnet152_fname = submissions_fpath.joinpath("XGBBERTResNet152.csv")
xgb_clip_fname = submissions_fpath.joinpath("XGBCLIP.csv")

sub_xgb_bert_resnet152 = pd.read_csv(xgb_bert_resnet152_fname, index_col=0, header=0)
sub_xgb_clip = pd.read_csv(xgb_clip_fname, index_col=0, header=0)

In [9]:
store_results(results, sub_xgb_bert_resnet152, "XGBBERTResNet152")
store_results(results, sub_xgb_clip, "XGBCLIP")
results

,,MSE,RMSE,MAE
Model,Run,,,
FPMeanBaselineModel,,37.629804,6.134314,4.201829
XGBBERTResNet152,,19.830486,4.453143,2.552919
XGBCLIP,,13.453174,3.667857,2.056221


### Core models for each run

In [10]:
models: List[FoodPricingBaseModel] = [
    FPCBOWResNet152ConcatBaselineModel,
    FPBERTResNet152ConcatModel,
    FPBERTResNet152WeightedConcatModel,
    FPCLIPConcatModel,
    FPCLIPWeightedConcatModel,
]

for model_class in models:
    for run in ["run_01", "run_02"]:
        name = model_class.__name__
        print(f"Calculating results for model {name} in run {run}")
        model = model_class.load_from_best_checkpoint(run=run)
        model.hparams.update({"store_submission_frame": False})
        submission_frame = model.make_submission_frame()
        
        store_results(results, submission_frame, name, run)

Calculating results for model FPCBOWResNet152ConcatBaselineModel in run run_01


Global seed set to 42


  0%|          | 0/34 [00:00<?, ?it/s]

Calculating results for model FPCBOWResNet152ConcatBaselineModel in run run_02


Global seed set to 42


...

Global seed set to 42


  0%|          | 0/34 [00:00<?, ?it/s]

Calculating results for model FPCLIPWeightedConcatModel in run run_02


Global seed set to 42


  0%|          | 0/34 [00:00<?, ?it/s]

In [11]:
results

MSE      RMSE       MAE
Model                              Run                                  
FPMeanBaselineModel                        37.629804  6.134314  4.201829
XGBBERTResNet152                           19.830486  4.453143  2.552919
XGBCLIP                                    13.453174  3.667857  2.056221
FPCBOWResNet152ConcatBaselineModel run_01  12.800708  3.577808   1.99431
                                   run_02  15.227577  3.902253  2.091247
FPBERTResNet152ConcatModel         run_01   9.899526  3.146351  1.597303
                                   run_02   9.823975  3.134322  1.523461
FPBERTResNet152WeightedConcatModel run_01   8.785368  2.964012  1.430603
                                   run_02   9.442002  3.072784  1.498697
FPCLIPConcatModel                  run_01   9.663844  3.108672  1.497017
                                   run_02  11.158245  3.340396  1.531476
FPCLIPWeightedConcatModel          run_01  10.076514  3.174352  1.563073
                                   run_02  10.715085   3.27339  1.623811

In [13]:
s = results.style.format(
    {
        ("MSE"): "{:.2f}",
        ("RMSE"): "{:.2f}",
        ("MAE"): "{:.2f}",
    }
)
with open("./tables/model_results.tex", mode="w") as f:
    s.to_latex(f)
s